In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm, tqdm_notebook
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('max_columns', 30, 'max_rows', 30)

In [2]:
train = pd.read_csv('boaz_train_0819.csv',encoding='cp949')
test = pd.read_csv('boaz_test_0819.csv',encoding='cp949')
y_train = pd.read_csv('boaz_y_0819.csv',encoding='cp949')

In [3]:
data = pd.concat([train, test], axis=0)

In [8]:
train.head()

,custid,sales_date,sales_time,str_nm,goodcd,brd_nm,corner_nm,pc_nm,part_nm,team_nm,buyer_nm,import_flg,tot_amt,dis_amt,net_amt,inst_mon,inst_fee
0,0,2000-06-25 00:00:00,1212,무역점,2116050008000,에스티로더,수입종합화장품,화장품,명품잡화,잡화가용팀,화장품,1,90000,9000,81000,3,0
1,0,2000-06-25 00:00:00,1242,무역점,4125440008000,시슬리,수입종합화장품,화장품,명품잡화,잡화가용팀,화장품,1,39000,3900,35100,1,0
2,0,2000-08-26 00:00:00,1810,본점,2116052008000,크리니크,수입종합화장품,화장품,잡화파트,잡화가용팀,화장품,1,175000,17500,157500,3,0
3,0,2000-08-26 00:00:00,1830,본점,4106430119900,듀퐁,수입의류,명품토탈,잡화파트,잡화가용팀,수입명품,1,455000,45500,409500,3,0
4,0,2000-09-03 00:00:00,1802,무역점,2139141008000,랑콤,수입종합화장품,화장품,명품잡화,잡화가용팀,화장품,0,100000,10000,90000,3,0


In [4]:
# sales date - 월, 요일, 주중/주말 특성 추가
data['sales_date'] = pd.to_datetime(data['sales_date'], format='%Y-%m-%d')
data['month'] = data['sales_date'].dt.month
data['weekday'] = data['sales_date'].dt.dayofweek
data['weekend'] = data['weekday'].apply(lambda x: 1 if (x==5 or x==6) else 0)

# sales time - 아침/ 낮/ 밤 구매 나누기
data['sales_time'] = data['sales_time'].apply(lambda x: int(str(x)[:2]) if len(str(x)) == 4 else int(str(x)[:1]))
data['sales_time_cat'] = data['sales_time'].apply(lambda x: 'mrn_sales' if (9 <= x <= 12) else ('aftn_sales' if (13 <= x <= 18) else 'knt_sales'))

# dis_amt - 할인율 특성 추가
data['dis_rate'] = data['dis_amt']/data['tot_amt']

In [5]:
# feautures 만들기

# 시간별 구매 횟수
freq_per_time = pd.pivot_table(index='custid',columns='sales_time',values='tot_amt',aggfunc=np.size, fill_value=0, data=data.query("tot_amt>0")[['custid','sales_time','tot_amt']]).reset_index()

# 퇴근전/후 구매 빈도, 비율
def get_off_work(x):
    if 9<=x<=17:
        return('bf_work')
    else:
        return('af_work')
    
data['sales_type'] = data['sales_time'].map(get_off_work)
sales_type = pd.pivot_table(index='custid',columns='sales_type',values='tot_amt',aggfunc=np.size, fill_value=0,data = data.query('tot_amt>0')[['custid','sales_type','tot_amt']]).reset_index()

features = pd.merge(freq_per_time, sales_type, on = 'custid')

features['af_work_rate'] = features['af_work']/(features['af_work'] + features['bf_work'])
features['bf_work_rate'] = features['bf_work']/(features['af_work'] + features['bf_work'])

# 아침구매/ 낮구매/ 밤구매
freq_sales_cat = pd.pivot_table(index='custid',columns='sales_time_cat',values='tot_amt',aggfunc=np.size, fill_value=0, data=data.query("tot_amt>0")[['custid','sales_time_cat','tot_amt']]).reset_index()
features = pd.merge(features, freq_sales_cat,on='custid')

# 구매 빈도 특성 추가
features['sales_freq'] = data.groupby('custid')['sales_date'].count()

# 주말 구매 비율
features['wd_sales_rate'] = data.groupby('custid')['weekend'].sum()/features['sales_freq']

# 자주 가는 월 평균
features['month_mean'] = data.groupby('custid')['month'].mean()

# 총, 최대, 최소, 평균 구매액
features['tot_sales_amt'] = data.groupby('custid')['net_amt'].sum()
features['max_amt'] = data.groupby('custid')['net_amt'].max()
features['min_amt'] = data.groupby('custid')['net_amt'].min()
features['mean_amt'] = round(data.groupby('custid')['net_amt'].mean(),2)

# 총 할인금액
features['tot_dis_amt'] = data.groupby('custid')['dis_amt'].sum()

# 최대, 평균 할인율
features['max_dis'] = data.groupby('custid')['dis_rate'].max()
features['mean_dis'] = data.groupby('custid')['dis_rate'].mean()

# 할인 받은 빈도, 비율
features['dis_freq'] = data.query("dis_rate>0").groupby('custid')['dis_rate'].count()
features['dis_freq_rate'] = features['dis_freq']/features['sales_freq']

# 환불 받은 빈도, 비율
refund_freq = data.query("tot_amt<0").groupby('custid')['tot_amt'].agg([('refund_freq','count')])
features = pd.merge(features, refund_freq, how='left',on='custid').fillna(0)
features['refund_freq_rate'] = features['refund_freq']/features['sales_freq']

# 할부 빈도, 비율/ 평균 할부개월
inst_ft = data.query("inst_mon != 1").groupby('custid')['inst_mon'].agg([('inst_freq','count'),('inst_avg','mean')])
features = pd.merge(features, inst_ft, how='left',on = 'custid').fillna(0)
features['inst_freq_rate'] = features['inst_freq']/features['sales_freq']

In [6]:
# 자주가는 지점
data['str_nm'] = data['str_nm'].apply(lambda x: 'mu' if x=='무역점' else('bon' if x=='본점' else('cheon' if x=='천호점' else 'sin')))

f = data.groupby(['custid','str_nm'])['str_nm'].agg([('str_freq','count')]).reset_index()

lst = []
for i in tqdm_notebook(f['custid'].unique()) : 
    f2 = f.query("custid == @i")
    lst.append(f2.sort_values(by='str_freq',ascending=False)['str_nm'].iloc[0])

features['str_freq'] = lst
features = pd.get_dummies(features, 'str_freq')

C:\anaconda3\lib\site-packages\ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


In [7]:
# 선호 브랜드
df = pd.merge(train, y_train, on='custid')
wo_prefer_brd = df.query("gender==0").brd_nm.value_counts().index[2:12].tolist()
ma_prefer_brd = df.query("gender==1").brd_nm.value_counts().index[2:12].tolist()

In [8]:
def map_wo_brd(x):
    if x == wo_prefer_brd[0]:
        x=10
    elif x == wo_prefer_brd[1]:
        x=9
    elif x == wo_prefer_brd[2]:
        x=8
    elif x == wo_prefer_brd[3]:
        x=7
    elif x == wo_prefer_brd[4]:
        x=6
    elif x == wo_prefer_brd[5]:
        x=5
    elif x == wo_prefer_brd[6]:
        x=4
    elif x == wo_prefer_brd[7]:
        x=3
    elif x == wo_prefer_brd[8]:
        x=2
    elif x == wo_prefer_brd[9]:
        x=1
    else:
        x=0
    
    return x

In [9]:
def map_ma_brd(x):
    if x == ma_prefer_brd[0]:
        x=10
    elif x == ma_prefer_brd[1]:
        x=9
    elif x == ma_prefer_brd[2]:
        x=8
    elif x == ma_prefer_brd[3]:
        x=7
    elif x == ma_prefer_brd[4]:
        x=6
    elif x == ma_prefer_brd[5]:
        x=5
    elif x == ma_prefer_brd[6]:
        x=4
    elif x == ma_prefer_brd[7]:
        x=3
    elif x == ma_prefer_brd[8]:
        x=2
    elif x == ma_prefer_brd[9]:
        x=1
    else:
        x=0
    
    return x

In [10]:
data['wo_brd_point'] = data['brd_nm'].apply(map_wo_brd)
data['ma_brd_point'] = data['brd_nm'].apply(map_ma_brd)

features['wo_brd_point'] = data.groupby('custid')['wo_brd_point'].mean()
features['ma_brd_point'] = data.groupby('custid')['ma_brd_point'].mean()

In [11]:
# 각 성별별 인기있는 상위 10개 제품의 구성비를 고객별로 평균냄
lst = df.query("gender==0").goodcd.value_counts().index[:10]
value_list = []

for i in range(10):
    a = lst[i]
    part = df.query("gender==0")[df.query("gender==0").goodcd == a].shape[0]
    total = df.query("gender==0").shape[0]
    value = round(part/ total,5)
    value_list.append(value)

value_array = np.array(value_list)
df1 = pd.DataFrame({'goodcd':lst, 'value_wo':value_array})
data = pd.merge(data, df1, how='left', on='goodcd')
data.fillna(9999, inplace=True)
features['value_wo'] = data.groupby('custid')['value_wo'].mean()

In [12]:
lst = df.query("gender==1").goodcd.value_counts().index[:10]
value_list = []

for i in range(10):
    a = lst[i]
    part = df.query("gender==1")[df.query("gender==1").goodcd == a].shape[0]
    total = df.query("gender==1").shape[0]
    value = round(part/ total,5)
    value_list.append(value)

value_array = np.array(value_list)
df2 = pd.DataFrame({'goodcd':lst, 'value_ma':value_array})
data = pd.merge(data, df2, how='left', on='goodcd')
data.fillna(9999, inplace=True)
features['value_ma'] = data.groupby('custid')['value_ma'].mean()

In [13]:
# part_nm 피쳐에 '여성/ 남성'이 들어간 컬럼 
data['include_wo_part'] = data['part_nm'].str.contains('여성').astype(int)
data['include_ma_part'] = data['part_nm'].str.contains('남성').astype(int)

features['ft_wo_part'] = data.groupby('custid')['include_wo_part'].sum()
features['ft_ma_part'] = data.groupby('custid')['include_ma_part'].sum()

In [14]:
# pc_nm 피쳐에 '여성/ 남성'이 들어간 컬럼 
data['include_wo_pc'] = data['pc_nm'].str.contains('여성').astype(int)
data['include_ma_pc'] = data['pc_nm'].str.contains('남성').astype(int)

features['ft_wo_pc'] = data.groupby('custid')['include_wo_pc'].sum()
features['ft_ma_pc'] = data.groupby('custid')['include_ma_pc'].sum()

In [15]:
# corner_nm 피쳐에 '여성/ 남성'이 들어간 컬럼 
data['include_wo_cor'] = data['corner_nm'].str.contains('여성').astype(int)
data['include_ma_cor'] = data['corner_nm'].str.contains('남성').astype(int)

features['ft_wo_cor'] = data.groupby('custid')['include_wo_cor'].sum()
features['ft_ma_cor'] = data.groupby('custid')['include_ma_cor'].sum()

In [16]:
# 세 가지 특성을 평균
features['ft_wo'] = (features['ft_wo_part'] + features['ft_wo_pc'] + features['ft_wo_cor'])/3
features['ft_ma'] = (features['ft_ma_part'] + features['ft_ma_pc'] + features['ft_ma_cor'])/3

In [17]:
features.drop(['ft_wo_part','ft_ma_part','ft_wo_pc','ft_ma_pc','ft_wo_cor','ft_ma_cor'], axis=1, inplace=True)

In [18]:
# 물품 특성, lgb를 위해 한글 컬럼명 변경
data['buyer_nm'] = data['buyer_nm'].str.replace('행사장.*','행사장')
data['buyer_nm'] = data['buyer_nm'].str.replace('피혁A','피혁')
data['buyer_nm'] = data['buyer_nm'].str.replace('피혁B','피혁')

data['buyer_nm'] = data['buyer_nm'].map({'화장품':'a', '수입명품':'b', '캐릭터캐주얼':'c', '트래디셔널캐주얼':'d', '유아동복':'e', '니트단품':'f', '영캐주얼':'g',
       '엘레강스캐주얼':'h', '가전':'i', '섬유':'j', '장신구':'k', '조리욕실':'l', '스포츠':'m', '침구수예':'n', '피혁':'o', '일반식품':'p',
       '유니캐주얼':'q', '정장셔츠':'r', '디자이너부띠끄':'s', '문화완구':'t', '타운모피':'u', '조리식품':'v', '기타바이어':'w',
       '도자기크리스탈':'x', '가구':'y', '생활용품':'z', '행사장':'aa', '청과곡물':'bb', '점외':'cc'})

buyer = pd.pivot_table(index='custid',columns='buyer_nm',values='tot_amt',aggfunc=np.size,fill_value=0,data=data.query("tot_amt>0")[['custid','buyer_nm','tot_amt']]).reset_index()
features = pd.merge(features, buyer, on='custid')

In [19]:
# 데이터 분할
X_train = pd.DataFrame({'custid':train['custid'].unique()})
X_train = pd.merge(X_train, features, how='left',on='custid')

y_train = y_train['gender']

X_test = pd.DataFrame({'custid':test['custid'].unique()})
X_test = pd.merge(X_test, features, how='left',on='custid')

In [30]:
# 랜덤 포레스트
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators= 350,min_samples_split = 2,min_samples_leaf = 5, max_depth=12, random_state=0)
rf.fit(X_train,y_train)
pred0 = rf.predict_proba(X_test)[:,1]

In [26]:
# XGBoost
from xgboost import XGBClassifier
import sys, warnings
if not sys.warnoptions: warnings.simplefilter("ignore")

xgb = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=-1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)
xgb.fit(X_train, y_train)
pred1 = xgb.predict_proba(X_test)[:,1]

In [27]:
# LightGBM
from lightgbm import LGBMClassifier

lgb = LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.018789800436355142,
        max_depth=58, min_child_samples=20, min_child_weight=0.001,
        min_split_gain=0.0, n_estimators=394, n_jobs=-1, num_leaves=31,
        objective=None, random_state=0, reg_alpha=0.0, reg_lambda=0.0,
        silent=True, subsample=0.6120957227224214,
        subsample_for_bin=200000, subsample_freq=0)

lgb.fit(X_train,y_train)
pred2 = lgb.predict_proba(X_test)[:,1]

In [33]:
# 일반 가중치 방법
pred = 0.3*pred0 + 0.4*pred1 + 0.3*pred2

In [34]:
fname = 'result4.csv'
submissions = pd.concat([pd.Series(test['custid'].unique(), name="custid"), pd.Series(pred, name="gender")] ,axis=1)
submissions.to_csv(fname, index=False)
print("'{}' is ready to submit." .format(fname))

'result4.csv' is ready to submit.
